# Data Analysis

## Libs & Helper Functions

In [20]:
import pandas as pd
from pandasql import sqldf
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go

In [21]:
def query_table(query_text):
    result = sqldf(query_text, globals())
    return result

## Datasets

In [22]:
df_user = pd.read_csv('business_data_analyst_tech_challenge_2023_h2_user_data.csv')
df_transactions = pd.read_csv('business_data_analyst_tech_challenge_2023_h2_transaction_data.csv')

In [23]:
dim_user = query_table('SELECT * FROM df_user')

fct_transactions = query_table('SELECT * FROM df_transactions')

dim_age_bin = query_table('''
WITH age_groups AS 
(
   SELECT
      age,
      NTILE(5) OVER (
   ORDER BY
      age) AS age_bin 
   FROM
      dim_user 
   WHERE
      age BETWEEN 17 AND 98 		--There are some users with age over 100 years, we are considering as typo
)
SELECT
   age_bin,
   COUNT(*) AS unique_users,
   CAST(MIN(age) AS INT) AS vl_age_bin_min,
   CAST(MAX(age) AS INT) AS vl_age_bin_max,
   CAST(MIN(age) AS INT) || '-' || CAST(MAX(age) AS INT) AS nm_age_bin 
FROM
   age_groups 
GROUP BY
   1
''')

In [26]:
obt_transactions = query_table('''
    SELECT
        f.*,
        u.country_code,
        u.total_claimed_wealth_usd,
        u.age,
        a.nm_age_bin
    FROM 
        fct_transactions AS f
    LEFT JOIN
        dim_user AS u
        ON f.user_id = u.user_id 
    LEFT JOIN
        dim_age_bin AS a
        ON u.age >= a.vl_age_bin_min 
        AND u.age < a.vl_age_bin_max 
''')

,user_id,timestamp,deposit_usd,withdrawal_usd,spot_trade_volume_usd,web3_thematic_purchase_usd,golden_thematic_purchase_usd,web3_thematic_sell_usd,golden_thematic_sell_usd,country_code,total_claimed_wealth_usd,age,nm_age_bin
0,af0cbfcbc797f44ae08bdcb9ad1e3365,2023-01-01,240.806465,266.091144,12.653927,NaN,NaN,NaN,NaN,GB,0_to_50k,41.0,35-45
1,c926e100b218db893d93032071c4409a,2023-01-01,129.224881,136.608073,3.691596,NaN,NaN,NaN,NaN,PL,0_to_50k,21.0,17-22
2,f2d1f504e16b5da35577da027ee77730,2023-01-01,50.655233,56.014557,2.684015,NaN,NaN,NaN,NaN,GB,50k_to_200k,37.0,35-45
3,1a8eee8df2bf99e5a018891c2958ef41,2023-01-01,30.791152,NaN,30.479976,NaN,NaN,NaN,NaN,FR,0_to_50k,25.0,22-28
4,e3619093914742a6c0ddbb34e9bc9729,2023-01-01,5666.924522,NaN,248.595810,NaN,NaN,NaN,NaN,FR,50k_to_200k,31.0,28-35
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77572,c254c3852fcbb89b7a6e0fe98a3c36c0,2023-08-14,48.720828,NaN,3094.448141,NaN,NaN,NaN,NaN,CH,0_to_50k,36.0,35-45
77573,7046a3dfe1b887489bf3af27faab2c5c,2023-08-14,882.744358,NaN,NaN,NaN,NaN,NaN,NaN,MT,200k_to_1m,28.0,28-35
77574,5b89d8fca257bf9734ea442ab8206c4e,2023-08-14,NaN,72.711829,NaN,NaN,NaN,NaN,NaN,DE,0_to_50k,55.0,45-97
77575,1e964c782583686e9efed8caa41bf51b,2023-08-14,353.232435,NaN,NaN,NaN,NaN,NaN,NaN,CH,0_to_50k,24.0,22-28


## EDA

In [27]:
obt_transactions.isnull().sum()

user_id                             0
timestamp                           0
deposit_usd                     35351
withdrawal_usd                  55834
spot_trade_volume_usd           18300
web3_thematic_purchase_usd      73873
golden_thematic_purchase_usd    76887
web3_thematic_sell_usd          76446
golden_thematic_sell_usd        77471
country_code                        0
total_claimed_wealth_usd            0
age                                 0
nm_age_bin                          3
dtype: int64

#### Splitting Variables

In [28]:
# Split dataset in numerics or categorical columns
num_attributes = obt_transactions.select_dtypes(include=['int64','float64'])
cat_attributes = obt_transactions.select_dtypes(exclude=['int64','float64','datetime64[ns]'])

In [29]:
num_attributes.describe().T

,count,mean,std,min,25%,50%,75%,max
deposit_usd,42226.0,4894.192345,27289.209158,0.000212,159.698758,552.301750,2336.372531,2.277356e+06
withdrawal_usd,21743.0,6334.320714,33904.203987,0.335861,182.111741,681.398460,3304.629653,2.284186e+06
spot_trade_volume_usd,59277.0,2025.010009,13018.526454,0.458010,56.140027,199.743963,790.966804,1.125254e+06
web3_thematic_purchase_usd,3704.0,378.935744,1186.277520,1.123460,32.161914,95.923625,242.340818,1.999156e+04
golden_thematic_purchase_usd,690.0,868.287149,3723.412282,9.859683,95.746582,182.442220,392.547715,4.459276e+04
web3_thematic_sell_usd,1131.0,501.569055,2022.388425,0.897963,31.767485,95.587679,289.430834,5.195406e+04
golden_thematic_sell_usd,106.0,1711.154838,12946.040305,11.303134,105.738116,173.424097,373.444531,1.334545e+05
age,77577.0,37.039174,13.314612,18.000000,26.000000,35.000000,46.000000,9.700000e+01


In [30]:
cat_attributes.describe().T

,count,unique,top,freq
user_id,77577,13766,d5bf6d0e4f0a9521d2a63ba316dab161,189
timestamp,77577,226,2023-07-13,697
country_code,77577,65,FR,20319
total_claimed_wealth_usd,77577,5,0_to_50k,54090
nm_age_bin,77574,5,45-97,21148


In [32]:
correlation = num_attributes.corr(method='pearson')

mask = np.triu(np.ones_like(correlation, dtype=bool))

correlation = correlation.mask(mask)

fig = go.Figure(data=go.Heatmap(
        z=correlation.values,
        x=correlation.columns,
        y=correlation.index,
        colorscale='algae', 
        colorbar=dict(title='Pearson Correlation')
))

for i in range(len(correlation.index)):
    for j in range(len(correlation.columns)):
        fig.add_annotation(x=correlation.columns[j], y=correlation.index[i],
                           text=str(round(correlation.values[i, j], 2)),
                           showarrow=False, font=dict(color='white'))

fig.update_layout(title='Pearson Correlation Heatmap', xaxis_title='Attributes', yaxis_title='Attributes')

fig.show()

# A. Case study

## Task 1

#### Q: Two weeks before the public release of the golden thematic, the responsible product manager reaches out to ask for an assessment on what they can expect in terms of adoption for this new product. Propose a set of metrics that you believe to be relevant, and create a business report that you could present to the product manager and the executive team that evaluates the possible business impact of the new feature. How would you define pessimistic/realistic/optimistic targets for the most relevant proposed metrics?

#### A: The answer to this question is available on file **_A1-Assessment of Adoption and Business Impact for Golden Thematic Product_** within this folder.

## Task 2

#### Q: During your initial discussion the product manager mentions several times that some user segmentation would be great, but they don’t have any clear ideas in mind. Take this into account during your exploration and in your presentation.

#### A: The current details available, such as Country, Age, and Claimed Wealth provide a initial step on identifying a cohort. However, additional information such as Gender, Education Level, Occupation, and Referral Source can improve our analysis, allowing deeper insights into user behavior and preferences.

## Task 3

#### Q: A week after the launch of the new golden thematic, the product manager comes back to you asking to evaluate the success of the release, as he finds it somewhat underwhelming. He also asks what can be done to increase adoption, and once again mentions a cohort analysis to identify better/worse performing user groups. Can you find any patterns that could help improve overall adoption? If yes, what actions can you propose? What (if any) additional support would you need to deliver on those?

#### A: The answer is available on file **_A3-First Week Adoption Analysis_**. The data used to create the file can be found below


### Data from First Week After Release

In [33]:
first_week_result_general = query_table('''
SELECT
   *
FROM
   obt_transactions
WHERE
   DATE(timestamp) BETWEEN '2023-07-18' AND '2023-07-25'
''')
first_week_result_general.head()

,user_id,timestamp,deposit_usd,withdrawal_usd,spot_trade_volume_usd,web3_thematic_purchase_usd,golden_thematic_purchase_usd,web3_thematic_sell_usd,golden_thematic_sell_usd,country_code,total_claimed_wealth_usd,age,nm_age_bin
0,ba335e2c58434dc9ba00f975b8746eb2,2023-07-18,417.151560,NaN,NaN,NaN,103.294787,NaN,NaN,RE,0_to_50k,38.0,35-45
1,b6d7ae84f86c6aedaa57024eeab8d109,2023-07-18,NaN,1268.315253,1251.825302,NaN,94.050153,NaN,NaN,CH,0_to_50k,34.0,28-35
2,9803493d0fa9149ce058e5c89f0c36e5,2023-07-18,NaN,NaN,119.684639,NaN,NaN,NaN,NaN,CH,0_to_50k,18.0,17-22
3,8f1ebb02908d869dfe549632a01f6a00,2023-07-18,NaN,NaN,NaN,NaN,664.944570,NaN,NaN,CA,0_to_50k,40.0,35-45
4,2b4bb079ac236da9fee73f8717b3a1d2,2023-07-18,14306.165719,NaN,7115.548663,NaN,NaN,NaN,NaN,PL,200k_to_1m,81.0,45-97


In [34]:
first_week_result_general.describe()

,deposit_usd,withdrawal_usd,spot_trade_volume_usd,web3_thematic_purchase_usd,golden_thematic_purchase_usd,web3_thematic_sell_usd,golden_thematic_sell_usd,age
count,1427.000000,933.000000,2421.000000,59.000000,455.000000,50.000000,41.000000,3271.000000
mean,3552.355660,4486.913263,1401.667347,294.187455,982.353017,274.974657,3628.615119,37.991440
std,9410.369388,15094.337737,5359.359551,1026.038180,4128.443092,552.219619,20791.721436,13.219272
min,0.124222,1.192684,1.747763,5.283260,13.307541,4.800224,11.303134,18.000000
25%,233.477336,170.550166,46.750026,23.821083,103.525473,33.035178,125.114561,27.000000
50%,702.217050,689.546334,165.789237,63.524706,192.569382,153.612799,180.610087,37.000000
75%,2537.830792,2744.183128,713.090202,171.980618,451.579324,287.316841,313.472169,47.000000
max,117911.446572,303426.240437,139884.620144,7781.793296,44592.758992,3772.185100,133454.527312,91.000000


In [35]:
first_week_result_golden_users = query_table('''
SELECT
   * 
FROM
   obt_transactions 
WHERE
   DATE(timestamp) BETWEEN '2023-07-18' AND '2023-07-25'
   AND (golden_thematic_purchase_usd IS NOT NULL 
   OR golden_thematic_sell_usd IS NOT NULL)
''')
first_week_result_golden_users.head()

,user_id,timestamp,deposit_usd,withdrawal_usd,spot_trade_volume_usd,web3_thematic_purchase_usd,golden_thematic_purchase_usd,web3_thematic_sell_usd,golden_thematic_sell_usd,country_code,total_claimed_wealth_usd,age,nm_age_bin
0,ba335e2c58434dc9ba00f975b8746eb2,2023-07-18,417.151560,NaN,NaN,NaN,103.294787,NaN,NaN,RE,0_to_50k,38.0,35-45
1,b6d7ae84f86c6aedaa57024eeab8d109,2023-07-18,NaN,1268.315253,1251.825302,NaN,94.050153,NaN,NaN,CH,0_to_50k,34.0,28-35
2,8f1ebb02908d869dfe549632a01f6a00,2023-07-18,NaN,NaN,NaN,NaN,664.944570,NaN,NaN,CA,0_to_50k,40.0,35-45
3,75adf154758d62a8d1e5f311250d946a,2023-07-18,639.968277,NaN,NaN,NaN,315.843463,NaN,NaN,FR,200k_to_1m,49.0,45-97
4,2f5073d2680db3e8b616350cb925cb8a,2023-07-18,NaN,NaN,235.598513,NaN,195.592855,NaN,NaN,FR,0_to_50k,29.0,28-35


In [36]:
first_week_result_golden_users.describe()

,deposit_usd,withdrawal_usd,spot_trade_volume_usd,web3_thematic_purchase_usd,golden_thematic_purchase_usd,web3_thematic_sell_usd,golden_thematic_sell_usd,age
count,173.000000,18.000000,154.000000,22.000000,455.000000,20.000000,41.000000,483.000000
mean,2497.204864,1210.523912,661.080829,168.775609,982.353017,432.117628,3628.615119,39.788820
std,7168.197074,2501.881538,2081.491345,273.701513,4128.443092,823.730202,20791.721436,13.127405
min,6.916558,31.719201,2.246668,5.283260,13.307541,4.800224,11.303134,18.000000
25%,290.116842,167.698905,34.533019,26.306605,103.525473,99.298830,125.114561,29.000000
50%,559.364395,324.380749,111.399543,77.446040,192.569382,180.948882,180.610087,39.000000
75%,1504.011582,801.535111,277.466763,142.019495,451.579324,437.975918,313.472169,49.000000
max,54793.296063,10624.347293,17630.752043,1219.863777,44592.758992,3772.185100,133454.527312,78.000000


### KPI

#### Users Engagement

In [37]:
users_engagement = query_table('''
WITH all_users AS 
(
   SELECT
      COUNT(DISTINCT user_id) AS all_users 
   FROM
      first_week_result_general 
)
,
golden_thematic_users AS 
(
   SELECT
      COUNT(DISTINCT user_id) AS gt_users 
   FROM
      first_week_result_golden_users
)
SELECT
   t.all_users AS all_users,
   g.gt_users AS golden_thematic_users,
   ROUND((CAST(g.gt_users AS float) / t.all_users), 3) AS engagement_rate 
FROM
   all_users t 
   CROSS JOIN
      golden_thematic_users g
''')

users_engagement

,all_users,golden_thematic_users,engagement_rate
0,1811,386,0.213


#### Users Transactions Frequency

In [55]:
frq_thematic_gold_users_transactions = query_table('''
SELECT
   ROUND(CAST(COUNT(user_id) AS float) / COUNT(DISTINCT user_id), 3) AS qt_users_transaction_frequency 
FROM
   first_week_result_golden_users 
''')
frq_thematic_gold_users_transactions

,qt_users_transaction_frequency
0,1.251


#### Total Transaction Volume

In [54]:
total_transaction_volume = query_table('''
SELECT
   SUM(golden_thematic_purchase_usd) AS vl_total_golden_thematic_purchase_usd 
FROM
   first_week_result_golden_users
''')

total_transaction_volume

,vl_total_golden_thematic_purchase_usd
0,446970.622735


#### Average Transaction Volume

In [50]:
average_transaction_volume = query_table('''
SELECT
   AVG(golden_thematic_purchase_usd) AS vl_average_golden_thematic_purchase_usd 
FROM
   first_week_result_golden_users
''')

average_transaction_volume

,vl_average_golden_thematic_purchase_usd
0,982.353017


### Univariate Analysis

#### Country Metrics Table

In [40]:
users_metrics_by_country = query_table(
    '''
WITH all_users AS 
(
   SELECT
      country_code,
      COUNT(DISTINCT user_id) AS vl_all_users 
   FROM
      first_week_result_general 
   GROUP BY
      1 
)
,
golden_users AS 
(
   SELECT
      country_code,
      COUNT(user_id) AS total_gt_users,
      COUNT(DISTINCT user_id) AS unique_gt_users,
      SUM(golden_thematic_purchase_usd) AS total_golden_thematic_purchase_usd,
      AVG(golden_thematic_purchase_usd) AS avg_golden_thematic_purchase_usd,
      SUM(golden_thematic_sell_usd) AS total_golden_thematic_sell_usd,
      ROUND(CAST(COUNT(user_id) AS float) / COUNT(DISTINCT user_id), 3) AS retention_rate 
   FROM
      first_week_result_golden_users 
   GROUP BY
      1
)
SELECT
   g.*,
   ROUND(CAST(g.unique_gt_users AS float) / vl_all_users, 3) AS engagement_rate 
FROM
   golden_users AS g 
   LEFT JOIN
      all_users AS a 
      ON g.country_code = a.country_code
   ORDER BY 3 DESC
''')  

users_metrics_by_country

,country_code,total_gt_users,unique_gt_users,total_golden_thematic_purchase_usd,avg_golden_thematic_purchase_usd,total_golden_thematic_sell_usd,retention_rate,engagement_rate
0,FR,176,145,89524.581066,549.230559,9499.552519,1.214,0.321
1,CH,144,108,70993.004939,518.197116,3180.564019,1.333,0.261
2,GB,64,53,37679.799079,607.738695,829.766965,1.208,0.102
3,DE,22,19,144946.449410,6588.474973,133644.658388,1.158,0.190
4,BE,14,13,6611.083037,508.544849,186.895827,1.077,0.213
5,CA,13,9,9900.368949,825.030746,677.832282,1.444,0.529
6,ES,8,7,7851.212647,981.401581,51.803377,1.143,0.233
7,AT,4,4,1980.843628,495.210907,NaN,1.000,0.308
8,IT,4,4,589.174580,147.293645,NaN,1.000,0.267
9,LU,5,4,38953.766679,7790.753336,NaN,1.250,0.444


#### Country Metrics Chart

In [41]:
users_metrics_by_country = users_metrics_by_country.sort_values(by='unique_gt_users', ascending=False)

users_metrics_by_country['cumulative_users'] = users_metrics_by_country['total_gt_users'].cumsum()

users_metrics_by_country['cumulative_percentage'] = (users_metrics_by_country['cumulative_users'] / users_metrics_by_country['total_gt_users'].sum()) * 100

fig = px.bar(users_metrics_by_country, x='country_code', y='total_gt_users', 
             title='Total Users per Country and Cumulative Percentage',
             labels={'country_code': 'Country Code', 'total_gt_users': 'Total Users'},
             text='total_gt_users', # Show labels on bars
             color_discrete_sequence=['green'])

fig.add_trace(px.line(users_metrics_by_country, x='country_code', y='cumulative_percentage').data[0])

fig.update_traces(yaxis='y', selector=dict(type='bar'))
fig.update_traces(yaxis='y2', selector=dict(type='scatter'))

fig.update_layout(yaxis=dict(title='Total Users', side='left', showgrid=False),
                  yaxis2=dict(title='Cumulative Percentage', side='right', overlaying='y', showgrid=False))

fig.show()

####  Claimed Wealth Metrics Table

In [42]:
users_metrics_by_weatlh = query_table(
    '''
WITH all_users AS 
(
   SELECT
      total_claimed_wealth_usd,
      COUNT(DISTINCT user_id) AS vl_all_users 
   FROM
      first_week_result_general 
   GROUP BY
      1 
)
,
golden_users AS 
(
   SELECT
      total_claimed_wealth_usd,
      COUNT(user_id) AS total_gt_users,
      COUNT(DISTINCT user_id) AS unique_gt_users,
      SUM(golden_thematic_purchase_usd) AS total_golden_thematic_purchase_usd,
      AVG(golden_thematic_purchase_usd) AS avg_golden_thematic_purchase_usd,
      SUM(golden_thematic_sell_usd) AS total_golden_thematic_sell_usd,
      ROUND(CAST(COUNT(user_id) AS float) / COUNT(DISTINCT user_id), 3) AS retention_rate 
   FROM
      first_week_result_golden_users 
   GROUP BY
      1
)
SELECT
   g.*,
   ROUND(CAST(g.unique_gt_users AS float) / vl_all_users, 3) AS engagement_rate 
FROM
   golden_users AS g 
   LEFT JOIN
      all_users AS a 
      ON g.total_claimed_wealth_usd = a.total_claimed_wealth_usd
            ''')

users_metrics_by_weatlh

,total_claimed_wealth_usd,total_gt_users,unique_gt_users,total_golden_thematic_purchase_usd,avg_golden_thematic_purchase_usd,total_golden_thematic_sell_usd,retention_rate,engagement_rate
0,0_to_50k,309,248,106913.787951,368.668234,8442.350923,1.246,0.203
1,1m_to_5m,16,10,72265.301926,4817.686795,553.648586,1.600,0.333
2,200k_to_1m,60,48,56699.446291,1012.490112,851.145269,1.250,0.261
3,50k_to_200k,97,79,198829.519484,2137.951822,138926.075113,1.228,0.218
4,5m_or_more,1,1,12262.567083,12262.567083,NaN,1.000,0.091


####  Claimed Wealth Metrics Charts

In [43]:
users_metrics_by_weatlh = users_metrics_by_weatlh.sort_values(by='unique_gt_users', ascending=False).round(2)

fig = px.bar(users_metrics_by_weatlh, 
             x='total_claimed_wealth_usd', 
             y='unique_gt_users',
             title='Total Users by Wealth Group',
             labels={'total_claimed_wealth_usd': 'Claimed Wealth', 'unique_gt_users': 'Total Users'},
             text='unique_gt_users',
             color_discrete_sequence=['green'])
fig.update_layout(yaxis=dict(title='Total Users', side='left', showgrid=False))

fig.show()

In [44]:
users_metrics_by_weatlh = users_metrics_by_weatlh.sort_values(by='total_golden_thematic_purchase_usd', ascending=False).round(2)

fig = px.bar(users_metrics_by_weatlh, 
             x='total_claimed_wealth_usd', 
             y='total_golden_thematic_purchase_usd',
             title='Total Purchase by Wealth Group',
             labels={'total_claimed_wealth_usd': 'Total Claimed Wealth (USD)', 'total_golden_thematic_purchase_usd': 'Total Purchase (USD)'},
             text='total_golden_thematic_purchase_usd',
             text_auto='$,.2f',
             color_discrete_sequence=['green'])
fig.update_layout(yaxis_tickprefix = '$', yaxis_tickformat = ',.2f')
fig.update_layout(yaxis=dict(title='Total Purchase (USD)', side='left', showgrid=False))

fig.show()

#### By Age Metrics Table

In [45]:
users_metric_by_age = query_table(
    '''
WITH all_users AS 
(
   SELECT
      nm_age_bin,
      COUNT(DISTINCT user_id) AS vl_all_users 
   FROM
      first_week_result_general
   GROUP BY
      1 
)
,
golden_users AS 
(
   SELECT
      nm_age_bin,
      COUNT(user_id) AS total_gt_users,
      COUNT(DISTINCT user_id) AS unique_gt_users,
      ROUND(CAST(COUNT(user_id) AS float) / COUNT(DISTINCT user_id), 3) AS retention_rate,
      SUM(golden_thematic_purchase_usd) AS total_golden_thematic_purchase_usd,
      AVG(golden_thematic_purchase_usd) AS avg_golden_thematic_purchase_usd,
      SUM(golden_thematic_sell_usd) AS total_golden_thematic_sell_usd 
   FROM
      first_week_result_golden_users
   GROUP BY
      1
)
SELECT
   g.*,
   ROUND(CAST(g.unique_gt_users AS float) / vl_all_users, 3) AS engagement_rate 
FROM
   golden_users AS g 
   LEFT JOIN
      all_users AS a 
      ON g.nm_age_bin = a.nm_age_bin   
''')

users_metric_by_age

,nm_age_bin,total_gt_users,unique_gt_users,retention_rate,total_golden_thematic_purchase_usd,avg_golden_thematic_purchase_usd,total_golden_thematic_sell_usd,engagement_rate
0,17-22,31,28,1.107,6947.099214,239.555145,394.467656,0.160
1,22-28,71,53,1.340,19696.641950,312.645110,1400.328184,0.171
2,28-35,88,73,1.205,46776.821894,570.449047,3939.234413,0.206
3,35-45,114,87,1.310,65051.008943,607.953355,1920.392596,0.197
4,45-97,179,145,1.234,308499.050734,1772.983050,141118.797042,0.274


#### By Age Metrics Chart

In [46]:
users_metric_by_age = users_metric_by_age.sort_values(by='nm_age_bin', ascending=True).round(2)

fig = px.bar(users_metric_by_age, 
             x='nm_age_bin', 
             y='unique_gt_users',
             title='Total Users by Age Group',
             labels={'total_claimed_wealth_usd': 'Total Claimed Wealth (USD)', 'nm_age_bin': 'Age Group'},
             text='unique_gt_users',
             color_discrete_sequence=['green'])
fig.update_layout(yaxis=dict(title='Total Users', side='left', showgrid=False))

fig.show()

In [47]:
users_metric_by_age = users_metric_by_age.sort_values(by='nm_age_bin', ascending=True).round(2)

fig = px.bar(users_metric_by_age, 
             x='nm_age_bin', 
             y='total_golden_thematic_purchase_usd',
             title='Total Purchase by Age Group',
             labels={'nm_age_bin': 'Age Group', 'total_golden_thematic_purchase_usd': 'Total Purchase (USD)'},
             text='total_golden_thematic_purchase_usd',
             text_auto='$,.2f',
             color_discrete_sequence=['green'])
fig.update_layout(yaxis_tickprefix = '$', yaxis_tickformat = ',.2f')
fig.update_layout(yaxis=dict(title='Total Purchase (USD)', side='left', showgrid=False))

fig.show()

### Multivariate Analysis

In [52]:
users_metric_by_all_groups = query_table(
    '''
WITH all_users AS 
(
   SELECT
      country_code,
      total_claimed_wealth_usd,
      nm_age_bin,
      COUNT(DISTINCT user_id) AS vl_all_users 
   FROM
      first_week_result_general
   GROUP BY
      1, 2, 3
),
golden_users AS (
SELECT
   country_code,
   total_claimed_wealth_usd,
   nm_age_bin,
   COUNT(user_id) AS total_gt_users,
   COUNT(DISTINCT user_id) AS unique_gt_users,
   ROUND(CAST(COUNT(user_id) AS float) / COUNT(DISTINCT user_id), 3) AS retention_rate,
   SUM(golden_thematic_purchase_usd) AS total_golden_thematic_purchase_usd,
   AVG(golden_thematic_purchase_usd) AS avg_golden_thematic_purchase_usd,
   SUM(golden_thematic_sell_usd) AS total_golden_thematic_sell_usd 
FROM
   first_week_result_golden_users
GROUP BY
   1, 2, 3
ORDER BY
   5 DESC
)

SELECT
   g.*,
   ROUND(CAST(g.unique_gt_users AS float) / vl_all_users, 3) AS engagement_rate 
FROM
   golden_users AS g 
   LEFT JOIN
      all_users AS a 
      ON g.country_code = a.country_code 
      AND g.total_claimed_wealth_usd = a.total_claimed_wealth_usd   
      AND g.nm_age_bin = a.nm_age_bin 
''')

users_metric_by_all_groups

,country_code,total_claimed_wealth_usd,nm_age_bin,total_gt_users,unique_gt_users,retention_rate,total_golden_thematic_purchase_usd,avg_golden_thematic_purchase_usd,total_golden_thematic_sell_usd,engagement_rate
0,FR,0_to_50k,45-97,28,25,1.120,15501.966004,553.641643,1334.514806,0.362
1,CH,0_to_50k,22-28,32,21,1.524,12508.979831,416.965994,349.537140,0.288
2,FR,0_to_50k,22-28,25,21,1.190,3729.460925,169.520951,40.609164,0.328
3,CH,0_to_50k,28-35,24,18,1.333,9939.839953,451.810907,379.428995,0.295
4,FR,50k_to_200k,45-97,18,17,1.059,11304.724495,628.040250,3526.935606,0.531
...,...,...,...,...,...,...,...,...,...,...
90,RO,0_to_50k,35-45,2,1,2.000,253.969163,253.969163,250.610567,0.333
91,RO,50k_to_200k,45-97,4,1,4.000,5368.558906,1342.139727,NaN,1.000
92,SE,200k_to_1m,45-97,2,1,2.000,806.964249,403.482125,NaN,1.000
93,SI,0_to_50k,35-45,2,1,2.000,1649.764509,824.882255,NaN,1.000
